# Breaking Hardware AES on CW305 FPGA

This tutorial is a continuation from [Tutorial CW305-1 Building a Project](http://wiki.newae.com/Tutorial_CW305-1_Building_a_Project). Here, we'll use our hardware setup to find a fixed secret key that the Artix FPGA is using for AES encryption. This tutorial relies on previous knowledge from [Using_CW-Analyzer_for_CPA_Attack](PA_CPA_1-Using_CW-Analyzer_for_CPA_Attack.ipynb), so make sure you understand how that attack works.

In [ ]:
import sys
import chipwhisperer as cw
from chipwhisperer.capture.targets.CW305 import CW305
import time
import numpy as np
from Crypto.Cipher import AES
import time
import matplotlib.pyplot as plt
import pickle



In [ ]:
scope.dis()
target.dis()

In [ ]:
N = 1000
start = time.time()
ssx = 40
freq = 10E5

In [ ]:
scope = cw.scope()
scope.gain.db = 34
#scope.gain.db = 25
#Testa olika gain
scope.adc.samples = 70*ssx
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"
scope.clock.clkgen_freq = freq * ssx #this sets sampling frequency.
scope.clock.adc_src = "clkgen_x4"

In [ ]:
target = cw.target(scope,
        cw.targets.CW305, bsfile=bsfile, force=True)

In [ ]:
bsfile = r'cw305_top.bit'

In [ ]:
bsfile = r'clx2_10764_fr_95917_90317_62777_40517.bit'

In [ ]:
target.vccint_set(1.0)
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)
target.pll.pll_outfreq_set(10E6, 1) #this sets frequency of outfreq. Hopefully this is used by the FPGA during AES.
target.clkusbautooff = True
target.clksleeptime = 1

Next we'll connect to the CW305 board. Here we'll need to specify our bitstream file to load as well as the usual scope and target_type arguments.

Pick the correct bitfile for your CW305 board (e.g. either '35t' or '100t'). By setting `force=False`, the bitfile will only be programmed if the FPGA is uninitialized (e.g. after powering up). Change to `force=True` to always program the FPGA (e.g. if you have generated a new bitfile).

In [ ]:
project_file = "projects/clx2_10764_fr_95917_90317_62777_40517.cwp"
project = cw.create_project(project_file, overwrite=True)

Next we set all the PLLs. We enable CW305's PLL1; this clock will feed both the target and the CW ADC. As explained [here](http://wiki.newae.com/Tutorial_CW305-1_Building_a_Project#Capture_Setup), **make sure the DIP switches on the CW305 board are set as follows**:
- J16 = 0
- K16 = 1

In [ ]:
# ensure ADC is locked:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

Occasionally the ADC will fail to lock on the first try; when that happens, the above assertion will fail (and on the CW-Lite, the red LED will be on). Simply re-running the above cell again should fix things.

## Trace Capture
Below is the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and appends our new trace to the `traces[]` list.

Because we're capturing 5000 traces, this takes a bit longer than the attacks against software AES implementations.

Note that the encryption result is read from the target and compared to the expected results, as a sanity check.

In [ ]:
from tqdm import tnrange, trange
import numpy as np
import time
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()

traces = []
textin = []
keys = []
textout = []

N = 30000  # Number of traces

# initialize cipher to verify DUT result:
key, text = ktp.next()
x=[218,56,43, 161, 159, 221, 35, 30, 208, 37, 232, 130, 117, 38, 247, 91]
arr = bytearray(x)
key=arr

cipher = AES.new(bytes(key), AES.MODE_ECB)
#textproj = cw.open_project("projects/Default")
textin = np.load("TextIn.npy")

for i in trange(N, desc='Capturing traces'):
    # run aux stuff that should come before trace here

    #garbage, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    
    
    #textin.append(text)
    ret = cw.capture_trace(scope, target, bytearray(textin[i]), key)
    #ret = cw.capture_trace(scope, target, text, key)
    if not ret:
        print("Failed capture")
        continue

    if list(ret.textout) != list(cipher.encrypt(bytearray(textin[i]))):
    #if list(ret.textout) != list(cipher.encrypt(text)):
        print("Incorrect encryption result!\nGot {}\nExp {}\nIndex of error was {}".format(ret.textout, list(textin[i]), i))
        continue
    
    keys.append(key)
    traces.append(ret.wave)
    textout.append(ret.textout)
    
    project.traces.append(ret)

project.save()
    

In [ ]:
project.export("projects/clx2_106971_fr_91942_92708_99330_43457.zip")

In [ ]:
plt.plot(np.array(traces[:1000]).T)
plt.show()

In [ ]:
ntraces = np.array(traces)
ntextin = np.array(textin)
ntextout = np.array(textout)

print(ntraces.shape, ntextin.shape, ntextout.shape)

In [ ]:
project.save()

In [ ]:
max_value = 0.45

traces = np.array(traces)
textin = np.array(textin)
textout = np.array(textout)

mask = np.any(traces > max_value, axis=1)
traces = traces[~mask]
#textin = textin[~mask]
#textout = textout[~mask]

traces = np.array(traces)
textin = np.array(textin)
textout = np.array(textout)

#plt.xlim([0, 2500])
#plt.plot(traces.T)
#plt.show()

In [ ]:
plt.plot(traces.T)
plt.show()